# Entity Matching using Neo4j - Error analysis

_Salomon Tetelepta, May 2nd 2024_
* Notebook to improve the model by enrich the graph
    * detect model numbers based on ngram rarity

### Install dependencies

In [2]:
!pip install neo4j python-dotenv langchain-community --quiet

%load_ext watermark
%watermark -p neo4j

### Imports

In [2]:
from dotenv import load_dotenv, find_dotenv, dotenv_values
from langchain_community.graphs import Neo4jGraph
from pathlib import Path
from sklearn.manifold import TSNE
from sklearn.metrics import PrecisionRecallDisplay
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from wordfreq import lossy_tokenize, tokenize, word_frequency

import json
import matplotlib.pyplot as plt
import neo4j
import numpy as np

import os
import pandas as pd
import pickle
import re

### Settings

In [3]:
# path settings
project_path = Path(os.getcwd()).parent
data_path = project_path / "data"
output_path = project_path / "output"

database = "abt-buy"

# load env settings
load_dotenv(project_path / ".env")

# reproducability
np.random.seed(42)

### 1. Load Data

In [4]:
os.listdir(data_path / 'abt-buy' / 'record_descriptions')

['2_buy.csv', '1_abt.csv']

In [5]:
# abt and buy records
df_abt = pd.read_csv(data_path / 'abt-buy' / 'record_descriptions' / '1_abt.csv', encoding='unicode_escape')
df_buy = pd.read_csv(data_path / 'abt-buy' / 'record_descriptions' / '2_buy.csv')

# matches - train and validation set
df_train = pd.read_csv(data_path / 'abt-buy' / 'gs_train.csv')
df_val = pd.read_csv(data_path / 'abt-buy' / 'gs_val.csv')
df_test = pd.read_csv(data_path / 'abt-buy' / 'gs_test.csv')

# merge records with matches
df_abt_merged = df_abt.merge(df_train, left_on='subject_id', right_on='source_id', how='right')
df_train_merged = df_buy.merge(df_abt_merged, left_on='subject_id', right_on='target_id', how='right')

df_train_merged.head(3)

,subject_id_x,name_x,description_x,manufacturer,price_x,subject_id_y,name_y,description_y,price_y,source_id,target_id,matching
0,207390654,Sony Handycam HDR-SR10 High Definition Digital...,16:9 - 2.7' Hybrid LCD,Sony,549.0,33161,Sony High Definition HDV Handycam Camcorder - ...,Sony High Definition HDV Handycam Camcorder - ...,NaN,33161,207390654,False
1,208085180,Pioneer DEH-2000MP Car Audio Player,"CD-RW - CD-Text, MP3, WMA, WAV - LCD - 4 - 200...",Pioneer,84.0,36258,D-Link Broadband Cable Modem - DCM202,D-Link Broadband Cable Modem - DCM202/ DOCSIS ...,79.00,36258,208085180,False
2,90125786,Sanus Wall/Ceiling Speaker Mount - WMS3S SILVER,Plastic - 8 lb,Sanus,NaN,17417,Sanus 13' - 30' VisionMount Flat Panel TV Silv...,Sanus 13' - 30' VisionMount Flat Panel TV Silv...,39.99,17417,90125786,False


### Connect to Neo4j

In [6]:
# connect to Neo4j
graph = Neo4jGraph(
    url=os.getenv('NEO4J_URL'),
    username=os.getenv('NEO4J_USER'),
    password=os.getenv('NEO4J_PASS')
)

# create database if does not exist
graph._database = "system"
query = f"CREATE DATABASE `{database}` IF NOT EXISTS"
graph.query(query)

# change to target database
graph._database = database
print("database:", graph._database)

# check nr nodes in the graph
graph.query("MATCH (n) RETURN count(n)")

database: abt-buy


[{'count(n)': 571211}]

### Create Ngram dataset

In [7]:
import nltk
from nltk.corpus import words
from nltk.util import ngrams
from nltk.tokenize import word_tokenize

from itertools import islice, tee
from collections import defaultdict

In [8]:
%%time
# Wall time: 11.2 s

run_cell = False
if run_cell == True:
    nltk.download('words')

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 10 µs


In [9]:
def get_ngrams(s, n):
    ngrams = list(zip(*(islice(seq, index, None) for index, seq in enumerate(tee(s, n)))))
    return ["".join(n) for n in ngrams]

In [10]:
nr_words = None
list_words = [w.lower() for w in words.words()[:nr_words]]

In [11]:
%%time
# Wall time: 5.99 s

nr_words = None
list_words = [w.lower() for w in words.words()[:nr_words]]

output = []
for n in np.arange(3, 7):
    dict_ngrams = defaultdict(int)
    dict_ngrams
    for w in list_words:
        for ngram in get_ngrams(w, n=n):
            dict_ngrams[ngram] += 1

    df_ngrams = pd.DataFrame([dict_ngrams]).T.sort_values(0, ascending=False).rename(columns={0: 'count'})
    df_ngrams['n'] = n
    output.append(df_ngrams)
df_ngrams_all = pd.concat(output)

CPU times: user 6.15 s, sys: 84.7 ms, total: 6.23 s
Wall time: 6.24 s


### Save to CSV

In [13]:
df_ngrams_all.reset_index(names='ngram').to_csv(output_path / 'ngrams.csv', index=False)
df_ngrams_all = pd.read_csv(output_path / 'ngrams.csv')

### Add ngrams to Graph

In [137]:
df_ngrams_all.head()

,ngram,count,n
0,ess,10976,3
1,ion,9704,3
2,ter,9546,3
3,ati,9473,3
4,ing,8951,3


#### Add unique constraint

In [ ]:
query = "CREATE CONSTRAINT ngram_value FOR (ngram:Ngram) REQUIRE ngram.value IS UNIQUE"
graph.query(query)

#### brute force ngrams to graph

In [150]:
%%time
# Wall time: 2h 42min 10s

run_cell = False
if run_cell:
    for i, (idx, row) in enumerate(df_ngrams_all.iterrows()):
        if i % 50_000 == 0:
            print(f"{i} / {len(df_ngrams_all):,}")
        query = f"MERGE (ngram:Ngram {{value: '{ row['ngram']}', count: {row['count']} }})"
        graph.query(query)

0 / 545,329
50000 / 545,329
100000 / 545,329
150000 / 545,329
200000 / 545,329
250000 / 545,329
300000 / 545,329
350000 / 545,329
400000 / 545,329
450000 / 545,329
500000 / 545,329
CPU times: user 12min 45s, sys: 1min 41s, total: 14min 27s
Wall time: 2h 42min 10s


### Split WordLower nodes into Ngrams

* Create relationships `(:WordLower)-[:HAS_[N]GRAM]->(:Ngram)`

In [30]:
query = """MATCH (w:WordLower) RETURN w.value AS wordlower"""
results = graph.query(query)
df_results = pd.DataFrame(results)
df_results.head()

,wordlower
0,sony
1,turntable
2,-
3,pslx350h
4,pslx350h/


In [45]:
for n in np.arange(3, 7):
    df_results.loc[:, f'{n}gram'] = df_results['wordlower'].apply(lambda x: get_ngrams(x, n))

In [66]:
df_results

,wordlower,3gram,4gram,5gram,6gram
0,sony,"[son, ony]",[sony],[],[]
1,turntable,"[tur, urn, rnt, nta, tab, abl, ble]","[turn, urnt, rnta, ntab, tabl, able]","[turnt, urnta, rntab, ntabl, table]","[turnta, urntab, rntabl, ntable]"
2,-,[],[],[],[]
3,pslx350h,"[psl, slx, lx3, x35, 350, 50h]","[pslx, slx3, lx35, x350, 350h]","[pslx3, slx35, lx350, x350h]","[pslx35, slx350, lx350h]"
4,pslx350h/,"[psl, slx, lx3, x35, 350, 50h, 0h/]","[pslx, slx3, lx35, x350, 350h, 50h/]","[pslx3, slx35, lx350, x350h, 350h/]","[pslx35, slx350, lx350h, x350h/]"
...,...,...,...,...,...
9711,mb942z/a,"[mb9, b94, 942, 42z, 2z/, z/a]","[mb94, b942, 942z, 42z/, 2z/a]","[mb942, b942z, 942z/, 42z/a]","[mb942z, b942z/, 942z/a]"
9712,pack-int,"[pac, ack, ck-, k-i, -in, int]","[pack, ack-, ck-i, k-in, -int]","[pack-, ack-i, ck-in, k-int]","[pack-i, ack-in, ck-int]"
9713,mb943z/a,"[mb9, b94, 943, 43z, 3z/, z/a]","[mb94, b943, 943z, 43z/, 3z/a]","[mb943, b943z, 943z/, 43z/a]","[mb943z, b943z/, 943z/a]"
9714,mate,"[mat, ate]",[mate],[],[]


In [74]:
%%time
# Wall time: 21min 22s

for i, (idx, row) in enumerate(df_results.iterrows()):
    if i % 1000 == 0:
        print(f"{i}/{len(df_results)}")
    
    for n in np.arange(3, 7):
        for ngram in row[f'{n}gram']:
            value = row['wordlower'].replace("'", "\\'")
            ngram = ngram.replace("'", "\\'")
            
            query = f"""
            MATCH (w:WordLower) WHERE w.value = '{value}'
            MERGE (ngram:Ngram {{value: '{ngram}'}})
            MERGE (w)-[:HAS_{n}GRAM]->(ngram)
            SET w:PROCESSED
            RETURN count(ngram)
            """
            graph.query(query)

0/9716
1000/9716
2000/9716
3000/9716
4000/9716
5000/9716
6000/9716
7000/9716
8000/9716
9000/9716
CPU times: user 2min 52s, sys: 23.5 s, total: 3min 16s
Wall time: 21min 22s


#### Label rare ngrams

In [78]:
# first, label ngrams that have a count attribute, which means they appeared in the vocabulary
query = "MATCH (n:Ngram) WHERE n.count > 0 SET n:Vocab"
graph.query(query)

# next, label rare ngrams, the ones that do not appear in the vocabulary
query = "MATCH (n:Ngram) WHERE NOT n:Vocab SET n:NoVocab"
graph.query(query)

[]

#### Examples

<h4>Can we find model IDs and other reference ids by checking for rare ngrams that words have in common?</h4>
<img src="../images/7.1-example-sharing-multiple-rare-3grams.jpg" width="600">

<h4>Example of matches that share a rare ngram </h4>
<img src="../images/7.2-example-sharing-multiple-rare-3grams.jpg" width="600">

<pre>MATCH p=(ngram:NoVocab)<-[]-(:WordLower)<-[:HAS_WORD_LOWER]-(:Name)<-[:HAS_NAME]-(i1:Item)-[:IS_MATCH]-(i2:Item)-[:HAS_NAME]->(:Name)-[:HAS_WORD_LOWER]->(:WordLower)-[]->(ngram) RETURN p LIMIT 10</pre>

<h4>Example of no matches that share a rare ngram </h4>
<img src="../images/7.2-example-sharing-multiple-rare-3grams.jpg" width="600">

<pre>MATCH p=(ngram:NoVocab)<-[]-(:WordLower)<-[:HAS_WORD_LOWER]-(:Name)<-[:HAS_NAME]-(i1:Item)-[:NO_MATCH]-(i2:Item)-[:HAS_NAME]->(:Name)-[:HAS_WORD_LOWER]->(:WordLower)-[]->(ngram) RETURN p LIMIT 10</pre>